# Python для анализа данных

## Бот в Telegram

### Сбор статистики по Коронавирусу

*Ян Пиле*

### Есть сайт с информацией о распространении вируса в мире
Я предлагаю достать оттуда информацию в виде Pandas dataframe а затем по запросу пользователя в телеграм выгружать ее

Импортируем requests и запишем адрес сайта со статистикой по COVID в переменную

In [36]:
import requests
from bs4 import BeautifulSoup

После чего через get-запрос достанем текст страницы со статистикой и распарсим ее с помощью BeautifulSoup

In [37]:
url = 'https://www.worldometers.info/coronavirus/'
website = requests.get(url).text
soup = BeautifulSoup(website, 'lxml')
# soup

In [38]:
soup.find_all('table')[1]

<table class="table table-bordered table-hover main_table_countries" id="main_table_countries_yesterday" style="width:100%;margin-top: 0px !important;display:none;">
<thead>
<tr>
<th width="1%">#</th>
<th width="100">Country,<br/>Other</th>
<th width="20">Total<br/>Cases</th>
<th width="30">New<br/>Cases</th>
<th width="30">Total<br/>Deaths</th>
<th width="30">New<br/>Deaths</th>
<th width="30">Total<br/>Recovered</th>
<th width="30">New<br/>Recovered</th>
<th width="30">Active<br/>Cases</th>
<th width="30">Serious,<br/>Critical</th>
<th width="30">Tot Cases/<br/>1M pop</th>
<th width="30">Deaths/<br/>1M pop</th>
<th width="30">Total<br/>Tests</th>
<th width="30">Tests/<br/>
<nobr>1M pop</nobr>
</th>
<th width="30">Population</th>
<th style="display:none" width="30">Continent</th>
<th width="30">1 Case<br/>every X ppl</th><th width="30">1 Death<br/>every X ppl</th><th width="30">1 Test<br/>every X ppl</th>
<th width="30">New Cases/1M pop</th>
<th width="30">New Deaths/1M pop</th>
<th w

На странице две таблицы: одна отвечает за информацию на сегодняшний день (под номером 0), а вторая - за информацию на вчерашний день. Достанем обе таблицы (они лежат между тегами \<\table\>\)

In [39]:
table = soup.find_all('table')[0]
# table

In [40]:
len(soup.find_all('table'))

3

Разобьем таблицу на строки (они лежат между тегами tr)

In [41]:
rows = table.find_all('tr')
rows

[<tr>
 <th width="1%">#</th>
 <th width="100">Country,<br/>Other</th>
 <th width="20">Total<br/>Cases</th>
 <th width="30">New<br/>Cases</th>
 <th width="30">Total<br/>Deaths</th>
 <th width="30">New<br/>Deaths</th>
 <th width="30">Total<br/>Recovered</th>
 <th width="30">New<br/>Recovered</th>
 <th width="30">Active<br/>Cases</th>
 <th width="30">Serious,<br/>Critical</th>
 <th width="30">Tot Cases/<br/>1M pop</th>
 <th width="30">Deaths/<br/>1M pop</th>
 <th width="30">Total<br/>Tests</th>
 <th width="30">Tests/<br/>
 <nobr>1M pop</nobr>
 </th>
 <th width="30">Population</th>
 <th style="display:none" width="30">Continent</th>
 <th width="30">1 Case<br/>every X ppl</th><th width="30">1 Death<br/>every X ppl</th><th width="30">1 Test<br/>every X ppl</th>
 <th width="30">New Cases/1M pop</th>
 <th width="30">New Deaths/1M pop</th>
 <th width="30">Active Cases/1M pop</th>
 </tr>,
 <tr class="total_row_world row_continent" data-continent="North America" style="display: none">
 <td></td>


Сначала достанем заголовки (между тегами th). Для примера достанем заголовок первой колонки.

In [42]:
rows[0].find_all('th')[1].get_text().strip()

'Country,Other'

Теперь мы можем достать содержимое ячеек таблицы (между тегами td). Для примера достанем первую запись первого по счету столбца

In [43]:
rows[1].find_all('td')[1].get_text().strip()

'North America'

Протестируем сборку на примере одной колонки, а потом пробежимся по их списку в цикле

In [44]:
col1 = []

col1.append(rows[0].find_all('th')[1].get_text().strip()) # отдельно добавляем заголовок

for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
    r = row.find_all('td') # находим все теги td для строки таблицы
    col1.append(r[1].get_text().strip()) # сохраняем данные в наш список

print(col1)

['Country,Other', 'North America', 'Asia', 'South America', 'Europe', 'Oceania', 'Africa', '', 'World', 'USA', 'India', 'Brazil', 'France', 'Germany', 'UK', 'Italy', 'S. Korea', 'Russia', 'Turkey', 'Spain', 'Vietnam', 'Japan', 'Argentina', 'Australia', 'Netherlands', 'Iran', 'Mexico', 'Colombia', 'Indonesia', 'Poland', 'Portugal', 'Ukraine', 'DPRK', 'Malaysia', 'Thailand', 'Austria', 'Israel', 'Belgium', 'Taiwan', 'Chile', 'South Africa', 'Canada', 'Czechia', 'Switzerland', 'Greece', 'Philippines', 'Peru', 'Denmark', 'Romania', 'Sweden', 'Iraq', 'Serbia', 'Bangladesh', 'Hungary', 'Slovakia', 'Jordan', 'Georgia', 'Ireland', 'Pakistan', 'Singapore', 'New Zealand', 'Norway', 'Kazakhstan', 'Hong Kong', 'Morocco', 'Bulgaria', 'Croatia', 'Finland', 'Lebanon', 'Cuba', 'Tunisia', 'Lithuania', 'Slovenia', 'Belarus', 'Nepal', 'UAE', 'Uruguay', 'Guatemala', 'Bolivia', 'Panama', 'Mongolia', 'Ecuador', 'Costa Rica', 'Latvia', 'Saudi Arabia', 'Azerbaijan', 'Paraguay', 'Sri Lanka', 'Kuwait', 'Bahrain

In [45]:
col1[:236][-1]

'MS Zaandam'

Чтобы еще раз проверить, запустим для другой колонки - TotalCases

In [46]:
col2 = []

col2.append(rows[0].find_all('th')[2].get_text().strip()) # отдельно добавляем заголовок

for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
    r = row.find_all('td') # находим все теги td для строки таблицы
    col2.append(r[2].get_text().strip()) # сохраняем данные в наш список

print(col2)

['TotalCases', '107,319,239', '163,272,705', '60,700,163', '208,958,593', '10,402,278', '12,421,661', '721', '563,075,360', '90,683,223', '43,672,155', '33,005,278', '32,548,947', '29,308,100', '22,883,995', '19,667,320', '18,602,109', '18,468,310', '15,180,444', '13,032,841', '10,756,254', '9,810,874', '9,426,171', '8,605,813', '8,253,854', '7,255,017', '6,301,645', '6,198,848', '6,116,347', '6,024,488', '5,266,454', '5,019,125', '4,769,330', '4,600,736', '4,550,924', '4,547,061', '4,472,124', '4,320,107', '4,132,429', '4,095,096', '3,998,466', '3,986,208', '3,943,792', '3,843,522', '3,843,142', '3,721,413', '3,691,213', '3,034,352', '2,946,174', '2,523,404', '2,390,684', '2,045,101', '1,991,031', '1,932,788', '1,801,993', '1,700,526', '1,662,299', '1,614,631', '1,543,741', '1,540,778', '1,462,257', '1,451,515', '1,310,370', '1,276,686', '1,242,828', '1,180,636', '1,160,253', '1,158,485', '1,125,720', '1,106,490', '1,087,030', '1,071,837', '1,052,574', '994,037', '980,590', '966,075',

А теперь пройдемся циклом по 9 колонкам, сгружая данные в список списков

In [47]:
field_list = []
for i in range(0,10):
    col = []
    col.append(rows[0].find_all('th')[i+1].get_text().strip()) # отдельно добавляем заголовок
    for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
        r = row.find_all('td') # находим все теги td для строки таблицы
        col.append(r[i+1].get_text().strip()) # сохраняем данные в наш список
    field_list.append(col)

А вот заголовок второго столбца

In [48]:
field_list[2][0]

'NewCases'

Остается превратить эту структуру в DataFrame. Для этого сначала превратим ее в словарь из списков, где ключами будут названия колонок.

In [49]:
d = dict()
for i in range(9):
    d[field_list[i][0]] = field_list[i][1:]
# d

А теперь сгружаем этот словарь в функцию pd.DataFrame и он превращается в искомый DataFrame

In [50]:
import pandas as pd 
df = pd.DataFrame(d)
df.head(10)

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical"
0,North America,"107,319,239","+36,334","1,492,852",+92,"100,461,111","+24,700","5,365,276","9,399"
1,Asia,"163,272,705","+112,744","1,442,124",+57,"157,480,673","+35,595","4,349,908","10,875"
2,South America,"60,700,163","+3,801","1,309,025",+3,"57,751,953","+12,162","1,639,185","10,447"
3,Europe,"208,958,593",,"1,861,748",,"198,401,775","+14,836","8,695,070","6,618"
4,Oceania,"10,402,278","+56,048","14,846",+86,"9,891,818","+10,527","495,614",169
5,Africa,"12,421,661",,"256,335",,"11,569,351",,"595,975","1,016"
6,,721,,15,,706,,0,0
7,World,"563,075,360","+208,927","6,376,945",+238,"535,557,387","+97,820","21,141,028","38,524"
8,USA,"90,683,223",,"1,046,613",,"86,137,546",,"3,499,064","4,077"
9,India,"43,672,155",,"525,519",,"43,011,874",,"134,762",698


In [51]:
df.head(8)

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical"
0,North America,"107,319,239","+36,334","1,492,852",+92,"100,461,111","+24,700","5,365,276","9,399"
1,Asia,"163,272,705","+112,744","1,442,124",+57,"157,480,673","+35,595","4,349,908","10,875"
2,South America,"60,700,163","+3,801","1,309,025",+3,"57,751,953","+12,162","1,639,185","10,447"
3,Europe,"208,958,593",,"1,861,748",,"198,401,775","+14,836","8,695,070","6,618"
4,Oceania,"10,402,278","+56,048","14,846",+86,"9,891,818","+10,527","495,614",169
5,Africa,"12,421,661",,"256,335",,"11,569,351",,"595,975","1,016"
6,,721,,15,,706,,0,0
7,World,"563,075,360","+208,927","6,376,945",+238,"535,557,387","+97,820","21,141,028","38,524"


Для удобства переименуем колонки с составными названиями.

In [52]:
df = df.rename(columns={'Country,Other': 'Country', 'Serious,Critical': 'SeriousCritical'})

In [53]:
df.head(8)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical
0,North America,"107,319,239","+36,334","1,492,852",+92,"100,461,111","+24,700","5,365,276","9,399"
1,Asia,"163,272,705","+112,744","1,442,124",+57,"157,480,673","+35,595","4,349,908","10,875"
2,South America,"60,700,163","+3,801","1,309,025",+3,"57,751,953","+12,162","1,639,185","10,447"
3,Europe,"208,958,593",,"1,861,748",,"198,401,775","+14,836","8,695,070","6,618"
4,Oceania,"10,402,278","+56,048","14,846",+86,"9,891,818","+10,527","495,614",169
5,Africa,"12,421,661",,"256,335",,"11,569,351",,"595,975","1,016"
6,,721,,15,,706,,0,0
7,World,"563,075,360","+208,927","6,376,945",+238,"535,557,387","+97,820","21,141,028","38,524"


А теперь сгрузим весь процесс, который мы проделали, в одну функцию. Ее аргумент tag = 0 - информация за сегодняшний день, a tag=1 - информация за вчерашний день.

In [54]:
def stat(tag = 0):
    url = 'https://www.worldometers.info/coronavirus/'
    website = requests.get(url).text
    soup = BeautifulSoup(website, 'lxml')
    table = soup.find_all('table')[tag]
    rows = table.find_all('tr')
    d = dict()
    for i in range(10):
        col = []
        key = rows[0].find_all('th')[i+1].get_text().strip()
        
        for row in rows[1:236]:
            r = row.find_all('td')
            col.append(r[i+1].get_text().strip())
        d[key] = col
    df = pd.DataFrame(d)
    df = df.rename(columns = {'Country,Other':'Country', 'Serious,Critical':'SeriousCritical'})
    return df

In [55]:
stat().head(10)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,Tot Cases/1M pop
0,North America,"107,319,239","+36,334","1,492,852",+92,"100,461,111","+24,700","5,365,276","9,399",
1,Asia,"163,302,566","+142,655","1,442,173",+106,"157,528,998","+83,920","4,331,395","10,875",
2,South America,"60,700,163","+3,801","1,309,025",+3,"57,751,953","+12,162","1,639,185","10,447",
3,Europe,"208,958,593",,"1,861,748",,"198,401,775","+14,836","8,695,070","6,618",
4,Oceania,"10,402,278","+56,048","14,846",+86,"9,891,818","+10,527","495,614",169,
5,Africa,"12,421,661",,"256,335",,"11,569,351",,"595,975","1,016",
6,,721,,15,,706,,0,0,
7,World,"563,105,221","+238,838","6,376,994",+287,"535,605,712","+146,145","21,122,515","38,524","72,241"
8,USA,"90,683,223",,"1,046,613",,"86,137,546",,"3,499,064","4,077",
9,India,"43,672,155",,"525,519",,"43,011,874",,"134,762",698,"31,029"


In [56]:
list(stat()['Country'][:8])

['North America',
 'Asia',
 'South America',
 'Europe',
 'Oceania',
 'Africa',
 '',
 'World']

В первых 8 строках таблицы в колонке Country лежат регионы + Мир. Одна строка пустая. Уберем ее с помощью спискового включения.

In [22]:
print(*list(i+'\n' for i in list(stat()['Country'][:8]) if i!=''))

North America
 Asia
 South America
 Europe
 Oceania
 Africa
 World



А с 8 по 220 строки лежат названия стран. Выведем ее с помощью спискового включения по 6 стран в линию (для этого вставим символ переноса строки)

In [57]:
x = stat()['Country'][8:236]
', '.join([e+'\n' if i%6 == 5 else e for i,e in enumerate(x)])

'USA, India, Brazil, France, Germany, UK\n, Italy, S. Korea, Russia, Turkey, Spain, Vietnam\n, Japan, Argentina, Australia, Netherlands, Iran, Mexico\n, Colombia, Indonesia, Poland, Portugal, Ukraine, DPRK\n, Malaysia, Thailand, Austria, Israel, Belgium, Taiwan\n, Chile, South Africa, Canada, Czechia, Switzerland, Greece\n, Philippines, Peru, Denmark, Romania, Sweden, Iraq\n, Serbia, Bangladesh, Hungary, Slovakia, Jordan, Georgia\n, Ireland, Pakistan, Singapore, New Zealand, Norway, Kazakhstan\n, Hong Kong, Morocco, Bulgaria, Croatia, Finland, Lebanon\n, Cuba, Tunisia, Lithuania, Slovenia, Belarus, Nepal\n, UAE, Uruguay, Guatemala, Bolivia, Panama, Mongolia\n, Ecuador, Costa Rica, Latvia, Saudi Arabia, Azerbaijan, Paraguay\n, Sri Lanka, Kuwait, Bahrain, Dominican Republic, Myanmar, Palestine\n, Estonia, Cyprus, Venezuela, Moldova, Egypt, Libya\n, Ethiopia, Honduras, Réunion, Armenia, Oman, Qatar\n, Bosnia and Herzegovina, Kenya, Zambia, Botswana, North Macedonia, Albania\n, Algeria, Lu

In [58]:
x = stat()['Country'][8:236]
import re
# print(x)
print(', '.join([e+'\n' if i%6 == 5 else e for i,e in enumerate(x)]).replace('\n,',',\n'))

USA, India, Brazil, France, Germany, UK,
 Italy, S. Korea, Russia, Turkey, Spain, Vietnam,
 Japan, Argentina, Australia, Netherlands, Iran, Mexico,
 Colombia, Indonesia, Poland, Portugal, Ukraine, DPRK,
 Malaysia, Thailand, Austria, Israel, Belgium, Taiwan,
 Chile, South Africa, Canada, Czechia, Switzerland, Greece,
 Philippines, Peru, Denmark, Romania, Sweden, Iraq,
 Serbia, Bangladesh, Hungary, Slovakia, Jordan, Georgia,
 Ireland, Pakistan, Singapore, New Zealand, Norway, Kazakhstan,
 Hong Kong, Morocco, Bulgaria, Croatia, Finland, Lebanon,
 Cuba, Tunisia, Lithuania, Slovenia, Belarus, Nepal,
 UAE, Uruguay, Guatemala, Bolivia, Panama, Mongolia,
 Ecuador, Costa Rica, Latvia, Saudi Arabia, Azerbaijan, Paraguay,
 Sri Lanka, Kuwait, Bahrain, Dominican Republic, Myanmar, Palestine,
 Estonia, Cyprus, Venezuela, Moldova, Egypt, Libya,
 Ethiopia, Honduras, Réunion, Armenia, Oman, Qatar,
 Bosnia and Herzegovina, Kenya, Zambia, Botswana, North Macedonia, Albania,
 Algeria, Luxembourg, Nigeria,

Для вывода результатов в текстовом формате воспользуемся библиотекой tabulate (в таком формате мы будем отправлять результаты в telegram'е)

In [59]:
!pip install tabulate

In [60]:
from tabulate import tabulate

Например общее количество случаев заболевания коронавирусом в Великобритании вывести можно так:

In [61]:
df[df['Country']=='UK'][['Country','TotalCases']].reset_index(drop=True)

,Country,TotalCases
0,UK,"22,883,995"


In [62]:
x = df[df['Country']=='UK'][['Country','TotalCases']].reset_index(drop=True)
print(tabulate(x, headers = x.columns, tablefmt="grid"))

+----+-----------+--------------+
|    | Country   | TotalCases   |
+====+===========+==============+
|  0 | UK        | 22,883,995   |
+----+-----------+--------------+


In [63]:
!pip install semidbm

In [64]:
import semidbm


In [65]:
db = semidbm.open('testdb', 'c')
db['foo'] = 'bar'
print(db['foo'].decode())
db.close()

bar


In [66]:
db = semidbm.open('testdb', 'c')
db['foo'].decode()

'bar'

In [67]:
db = semidbm.open('testdb', 'c')
del db['foo']

In [68]:
db['foo']

KeyError: b'foo'

In [ ]:
db.close()